In [1]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from urllib.request import urlopen
import urllib

from selenium import webdriver
import openpyxl
import time
import math
from collections import Counter


from tqdm.notebook import tqdm


In [2]:
def url_split(url):
    main_,para_1 = url.split("?")
    para_2 = para_1.split("&")
    para_3 = [d.replace("=",":").split(":") for d in para_2]
    params_ = {}
    for key,value in para_3:
        params_[key] = value
    return main_,params_

In [3]:
ua = UserAgent()

In [4]:
## 일단 상품평 많은 순으로 상위 60개만 긁어오기

def Main_page_SSG(Key_):
    header = {'User-Agent':ua.chrome,
              "referer":"https://with.gsshop.com/shop/search/main.gs"}

    url = "https://with.gsshop.com/shop/search/mainJson.gs?tq={}&rq=&cls=&eh=eyJzb3J0VHlwZSI6IlJFVklFV19DT1VOVCIsInNlbGVjdGVkIjoib3B0LXNvcnQifQ%3D%3D&lseq=395203 "

    html = requests.get(url.format(Key_),headers=header).text
    soup = BeautifulSoup(html,"html.parser")


    name_head,url_head,reveiw_count_1 = [],[],[]
    for tag in soup.select("li"):
        url_head.append(tag.find("a")["href"])
        name_head.append(tag.find("dt",class_="prd-name").text.strip().replace("\n","").replace("\t",""))
        reveiw_count_1.append(tag.find("dd",class_="user-side").find("button").text.replace("상품평","").replace(" ","").replace(",",""))
    Head_df = pd.DataFrame([name_head,reveiw_count_1,url_head]).T.rename(columns={0:"Name",1:"Review_count",2:"Url"})
    return Head_df

In [5]:
head_df = Main_page_SSG("헤어드라이기")
head_df

,Name,Review_count,Url
0,[TV상품]다이슨 에어랩 볼륨앤쉐이프,찜하기,http://www.gsshop.com/prd/prd.gs?prdid=6759532...
1,[TV상품]프리미엄라인 순수 골드라벨 염색제 + 살롱 드라이기,찜하기,http://www.gsshop.com/prd/prd.gs?prdid=5803382...
2,박준뷰티랩 식스모션 헤어 드라이기 PJP-3500 / 거치대 증정,4286,http://www.gsshop.com/prd/prd.gs?prdid=1055079...
3,JMW M5001A PLUS 전문가용 항공모터 드라이기,4124,http://www.gsshop.com/prd/prd.gs?prdid=6526506...
4,[14%할인+거치홀더]JMW 팬텀 로즈골드 MS6001A 항공모터드라이기,2787,http://www.gsshop.com/prd/prd.gs?prdid=2539903...
5,JMW 터보 항공기 드라이기 MS6001A 팬텀 펄화이트,2736,http://www.gsshop.com/prd/prd.gs?prdid=5291306...
6,[결제시14%할인]JMW BLDC항공모터 드라이기 MG1800 화이트,2392,http://www.gsshop.com/prd/prd.gs?prdid=2514706...
7,디자이너스 프리미엄 화이트 골드 헤어 드라이기 DGCB-004,1796,http://www.gsshop.com/prd/prd.gs?prdid=1318716...
8,[시크릿특가] JMW 에어젯 터보 MS6020B 항공모터 헤어 드라이기,1483,http://www.gsshop.com/prd/prd.gs?prdid=3468330...
9,[TV상품]순수 디어뮤즈 염색제 10박스 + 순수 살롱 드라이기,찜하기,http://www.gsshop.com/prd/prd.gs?prdid=6574101...


In [6]:
Ta_ = 0 

url1 = head_df["Url"][Ta_]
product_name = head_df["Name"][Ta_]
main,para = url_split(url1)
pro_id = para["prdid"]
pro_id

'67595326'

In [13]:
def First_id(head_df,Ta_):
    url1 = head_df["Url"][Ta_]
    product_name = head_df["Name"][Ta_]
    try:
        max_r  = int(head_df["Review_count"][Ta_])
    except:
        Ta_2 = int(Ta_)
        max_r = None
        while max_r is None:
            Ta_2 = Ta_2 +1
            try:
                max_r  = int(head_df["Review_count"][Ta_2])
            except:
                 pass
    
    main,para = url_split(url1)
    pro_id = para["prdid"]
    header = {'User-Agent':ua.chrome,
          "referer":"https://gsshop.com/"}
    main_2 = 'http://www.gsshop.com/mi15/knownew/revw/revwList.gs'
    para_2 = { 'contentDisNoYn': 'N'}
    para_2["prdid"] = pro_id
    html = requests.get(main_2,headers=header,params=para_2).text
    soup = BeautifulSoup(html,"html.parser")
    first_meid = 0
    for tag in soup.select("#reveiw-list"):
        first_meid = tag.find_all("article")[0]["data-messageid"]
    return first_meid,product_name,pro_id,max_r

In [16]:
First_id(head_df,3)

('163700240', 'JMW M5001A PLUS 전문가용 항공모터 드라이기', '65265063', 4124)

In [24]:
def Extraction(pro_id,first_meid):
    header = {'User-Agent':ua.chrome,
          "referer":"https://gsshop.com/"}
    
    main_3 = 'http://www.gsshop.com/mi15/knownew/revw/page/revwList.gs'
    para_3 = {}

    para_3["prdid"] = pro_id
    para_3['$messageId'] = first_meid
    html = requests.get(main_3,headers=header,params=para_3).text
    soup = BeautifulSoup(html,"html.parser")

    data_all = []
    for tag in soup.select("article"):
        header_ = tag.find("header").find("div",class_="write-info")
        date_ = header_.find("span",class_="date").text.replace("작성일","")
        id_ = header_.find("span",class_="writer").text.replace("작성자","")
        item_ = product_name
        score_ =int(header_.find("span",class_="star-rate").find("em")["style"].replace("width:","").replace("%",""))/20
        contents_ = tag.find("main").find("p").text
        no_ = int(tag.find("footer").find_all("button")[0]["data-rcmdcnt"])
        li_ = int(tag.find("footer").find_all("button")[1]["data-rcmdcnt"])
        channel_ = "GS"
        if li_ == 0:
            liked_ = 0
        else:
            liked_ = (li_ - no_)
        all_ = [date_,id_,item_,score_,contents_,liked_,channel_]
        data_all.append(all_)
    df_ = pd.DataFrame(data_all).rename(columns={0:"date",1:"id",2:"item",3:"score",4:"contents",5:"liked",6:"channel"})
    
    next_id = 0
    for tag in soup.select("div.next"):
        next_id = tag["data-messageid"]
        
    return df_,next_id

In [25]:
Extraction(pro_id,161684680)[1]

'161680494'

In [26]:
Extraction(pro_id,"161516758")[1]

'161464014'

In [27]:
def Crawling_all_GS(head_df,Ta_,loop_):
    fi,name_,pro_id,maxn = First_id(head_df,Ta_)
    review_all,nid_ = Extraction(pro_id,fi)
    #print(maxn)
    try:
        loop_ = (maxn//10)
        #print(loop_)
    except:
        pass
    tq = tqdm(range(loop_))

    for i in tq:
        review_candi,nid_ = Extraction(pro_id,nid_)
        review_all = pd.concat([review_all,review_candi]).reset_index(drop=True)
    return review_all,name_

In [28]:
test_,tdf_ = Crawling_all_GS(head_df,50,1)
test_

  0%|          | 0/10 [00:00<?, ?it/s]

,date,id,item,score,contents,liked,channel
0,2021.07.08,cou******,[TV상품]다이슨 에어랩 볼륨앤쉐이프,5.00,"추천할만하게 좋네요. 바람이 쎄지만 소음은 심하지 않고, 빨리 말릴 수 있습니다",0,GS
1,2021.07.03,js3****,[TV상품]다이슨 에어랩 볼륨앤쉐이프,3.00,가볍긴한데 드라이기 헤드가 헐렁해서 자꾸 빠지니까 짜증나서 잘안씁니다ㅠㅠ,0,GS
2,2021.06.30,c0**********@gmail.com,[TV상품]다이슨 에어랩 볼륨앤쉐이프,5.00,가볍고 색깔도 이쁘고요.바람이 써서 머리 빨리 말릴수 있어서 넘 좋아요.,0,GS
3,2021.06.30,pjs****,[TV상품]다이슨 에어랩 볼륨앤쉐이프,3.00,가격대비 여러가지가 사용하기에 좋아요~~,0,GS
4,2021.06.27,pur******,[TV상품]다이슨 에어랩 볼륨앤쉐이프,5.00,처음쓸때 고무탄내? 전기탄내가나서 교환하려했더니 3일정도 써보라고해서 써보고 지켜봤...,0,GS
...,...,...,...,...,...,...,...
104,2018.04.27,yun*******,[TV상품]다이슨 에어랩 볼륨앤쉐이프,5.00,소리는조금크지만 만족합니다,0,GS
105,2018.04.18,dem******,[TV상품]다이슨 에어랩 볼륨앤쉐이프,5.00,기존 낮은 와트 제품 이용하다 센것으로 사용하니 빠르고 좋으네요..맘에 듭니다~,2,GS
106,2018.04.17,jew********,[TV상품]다이슨 에어랩 볼륨앤쉐이프,5.00,좋아요.좋아요,0,GS
107,2018.04.13,soj*****,[TV상품]다이슨 에어랩 볼륨앤쉐이프,4.75,기존에도 유닉스를 사용해서 이번에도 유닉스꺼로 구매햇습니다~~ \r전에 사용하던것보...,0,GS


In [29]:
def Save_GS(head_df,path_,loop_):
    for i_ in range(len(head_df)):
        print("In {} Pro From {}".format(i_+1,len(head_df)+1))
        print()
        print(head_df["Name"][i_])
        out_,NA_ = Crawling_all_GS(head_df,i_,loop_)
        NA_ = NA_.replace("/","")
        out_.to_excel(path_.format(NA_))

In [30]:
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\GS\{}.xlsx"
Save_GS(head_df,path_,100)

In 1 Pro From 61

[TV상품]다이슨 에어랩 볼륨앤쉐이프


  0%|          | 0/428 [00:00<?, ?it/s]

In 2 Pro From 61

[TV상품]프리미엄라인 순수 골드라벨 염색제 + 살롱 드라이기


  0%|          | 0/428 [00:00<?, ?it/s]

In 3 Pro From 61

박준뷰티랩 식스모션 헤어 드라이기 PJP-3500 / 거치대 증정


...

  0%|          | 0/8 [00:00<?, ?it/s]

In 59 Pro From 61

다이슨 슈퍼소닉 헤어 드라이기 실리콘 커버 케이스


  0%|          | 0/8 [00:00<?, ?it/s]

In 60 Pro From 61

비달사순 2,000W 강력 헤어드라이기/헤어드라이어 VSD120K


  0%|          | 0/7 [00:00<?, ?it/s]